# Trade Execution

Executes the trade plan generated by `01_analysis_pipeline.ipynb`.
Verifies account state before and after execution.

| Step | Description |
|------|-------------|
| 1 | **Verify State** — confirm account matches notebook 1 snapshot |
| 2 | **Execute** — load trades, edit instructions, place orders |
| 3 | **Verify Execution** — confirm positions match expected state |

**Pre-requisite:** Run `01_analysis_pipeline.ipynb` first. It cleans up
the account (cancels orphan orders, covers shorts) and writes
`portfolio_state.json` — the contract that Step 1 checks.

**Safety checks (built-in):**
- Cancels duplicate orders before placing new ones
- Blocks any sell that would create a short position
- Caps buys to available cash (never goes negative)
- LIMIT GTC orders for outside-hours submission
- 10% trailing stops on all BUY fills

Output: `~/trading/live_portfolio/execution_log.csv`

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

import nest_asyncio
import pandas as pd
nest_asyncio.apply()

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))
sys.path.insert(0, str(PROJECT_ROOT / "notebooks"))

LIVE_DIR = Path.home() / "trading" / "live_portfolio"
TRADE_PLAN_FILE = LIVE_DIR / "trade_plan.csv"
STATE_FILE = LIVE_DIR / "portfolio_state.json"

IB_HOST = "127.0.0.1"
IB_PORT = 4001
IB_CLIENT_ID = 15
TRAILING_STOP_PCT = 10

# ── Limit Order Mode ─────────────────────────────────────
# Set True to use LIMIT GTC orders (submit outside market hours)
# Set False to use MARKET orders (must run during RTH 9:30-16:00 ET)
USE_LIMIT_ORDERS = True      # Recommended for evening/weekend submission
LIMIT_BUFFER_PCT = 1.0       # 1% buffer: BUY at close+1%, SELL at close-1%

print(f"Live dir:    {LIVE_DIR}")
print(f"State file:  {STATE_FILE}")
print(f"Trade plan:  {TRADE_PLAN_FILE}")
print(f"Order mode:  {'LIMIT GTC' if USE_LIMIT_ORDERS else 'MARKET'}")

---
## Step 1: Verify State

Connects read-only to IB and confirms the account matches the snapshot
taken by notebook 1 (`portfolio_state.json`):

- Same positions (exact share counts)
- Cash within 2% tolerance (small changes from dividends/interest)
- No open orders (cleanup already ran)
- No short positions
- `trade_plan.csv` checksum matches (file not tampered)

**If FAILED → re-run `01_analysis_pipeline.ipynb` before proceeding.**

In [ ]:
from scripts.s7_execute import verify_portfolio_state

verification = verify_portfolio_state(
    state_file=STATE_FILE,
    ib_host=IB_HOST,
    ib_port=IB_PORT,
    ib_client_id=IB_CLIENT_ID + 1,  # Read-only client ID
)

if not verification["valid"]:
    critical = [d for d in verification["discrepancies"] if d["severity"] == "CRITICAL"]
    print(f"\n*** STOP: {len(critical)} critical issue(s) found ***")
    print("Re-run 01_analysis_pipeline.ipynb to refresh the state.")
else:
    print("\nState verified — safe to proceed to Step 2.")

---
## Step 2: Execute Trades

Three sub-steps:
- **2a** Load trade plan from CSV
- **2b** Optional custom instruction edits + interpret
- **2c** Execute with `CONFIRM=True` gate

**Order modes:**
- `USE_LIMIT_ORDERS = True` (default): LIMIT GTC orders — submit anytime, fills at market open. BUY limits = close + 1%, SELL limits = close - 1%.
- `USE_LIMIT_ORDERS = False`: MARKET orders — must run during RTH (9:30-16:00 ET).

Trailing stops (10% TRAIL, GTC) are placed automatically on every BUY fill/submission.

In [ ]:
# Step 2a: Load trade plan
from scripts.s7_execute import load_trade_plan

trades = load_trade_plan(TRADE_PLAN_FILE)
print(f"Loaded {len(trades)} trades from {TRADE_PLAN_FILE}")
print(f"Order mode: {'LIMIT GTC' if USE_LIMIT_ORDERS else 'MARKET'}")
if USE_LIMIT_ORDERS:
    print(f"Limit buffer: {LIMIT_BUFFER_PCT}% from last close")
pd.DataFrame(trades)

In [ ]:
# Step 2b: Custom instructions + interpret
custom_instructions = {
    # "BND": "SKIP",
    # "DIM": "reduce to 30 shares",
}

from scripts.s7_execute import apply_custom_instructions, interpret_trades

trades = apply_custom_instructions(trades, custom_instructions)
final_trades = interpret_trades(trades)
print(f"{len(final_trades)} trades ready for execution")
pd.DataFrame(final_trades)

In [ ]:
# Step 2c: Execute
# ============================================
CONFIRM = Reue  # Set True to place REAL orders
# ============================================

from scripts.s7_execute import execute_trades

exec_results = execute_trades(
    final_trades, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
    trailing_stop_pct=TRAILING_STOP_PCT,
    confirm=CONFIRM,
    use_limit_orders=USE_LIMIT_ORDERS,
    limit_buffer_pct=LIMIT_BUFFER_PCT,
)

if exec_results:
    pd.DataFrame(exec_results)

---
## Step 3: Verify Execution

Run **after market hours** once all orders should have filled.

Compares actual IB positions against the `expected_post_trade` state from
`portfolio_state.json`. Reports:
- Matched positions (expected = actual)
- Mismatches (wrong share count)
- Unexpected positions (not in plan)
- Pending non-TRAIL orders (LIMIT GTC not yet filled)
- Cash comparison (5% tolerance since fill prices vary)

If incomplete, use the fix cell below to cancel stale orders and resubmit.

In [ ]:
from scripts.s7_execute import verify_execution

exec_check = verify_execution(
    state_file=STATE_FILE,
    ib_host=IB_HOST,
    ib_port=IB_PORT,
    ib_client_id=IB_CLIENT_ID + 1,
)

if exec_check["complete"]:
    print("\nAll trades executed as expected. Portfolio is on target.")
else:
    print("\nExecution incomplete — run the fix cell below.")
    if exec_check["mismatches"]:
        print(f"  {len(exec_check['mismatches'])} position mismatches")
    if exec_check["pending_orders"]:
        print(f"  {len(exec_check['pending_orders'])} orders still pending")

In [ ]:
# Step 3b: Fix unfilled orders
#
# Uses check_order_status to identify pending/missing trades,
# then cancels stale orders and resubmits with fresh prices.

# ============================================
CONFIRM_FIX = False  # Set True to cancel + resubmit
# ============================================

from scripts.s7_execute import check_order_status, fix_unfilled_orders

check = check_order_status(
    TRADE_PLAN_FILE, LIVE_DIR,
    ib_host=IB_HOST, ib_port=IB_PORT,
    ib_client_id=IB_CLIENT_ID + 1,
)

# Show status
if check["filled"]:
    print("\nFILLED:")
    df_filled = pd.DataFrame(check["filled"])[["action", "ticker", "shares", "fill_status", "fill_price"]]
    display(df_filled.style.set_properties(**{"background-color": "#d4edda"}))

if check["pending"]:
    print("\nPENDING (still working):")
    df_pending = pd.DataFrame(check["pending"])[["action", "ticker", "shares", "ib_status", "limit_price", "filled_qty", "remaining"]]
    display(df_pending.style.set_properties(**{"background-color": "#fff3cd"}))

if check["missing"]:
    print("\nMISSING (needs resubmission):")
    df_missing = pd.DataFrame(check["missing"])[["action", "ticker", "shares", "current_position"]]
    display(df_missing.style.set_properties(**{"background-color": "#f8d7da"}))

# Fix if confirmed
if check["pending"] or check["missing"]:
    fix_results = fix_unfilled_orders(
        check,
        TRADE_PLAN_FILE, LIVE_DIR,
        ib_host=IB_HOST, ib_port=IB_PORT, ib_client_id=IB_CLIENT_ID,
        trailing_stop_pct=TRAILING_STOP_PCT,
        use_limit_orders=USE_LIMIT_ORDERS,
        limit_buffer_pct=LIMIT_BUFFER_PCT,
        confirm=CONFIRM_FIX,
    )
    if fix_results:
        pd.DataFrame(fix_results)
else:
    print("\nNothing to fix — all trades completed.")

---

**Workflow:**
1. Run `01_analysis_pipeline.ipynb` — cleans account, generates trade plan, writes `portfolio_state.json`
2. Run Step 1 — verify state matches
3. Run Step 2 with `CONFIRM=False` — dry run preview
4. Set `CONFIRM=True` — places LIMIT GTC orders
5. After market hours: Run Step 3 — verify fills
6. If anything pending/missing: set `CONFIRM_FIX=True` — cancel stale + resubmit
7. Repeat Steps 3 + fix until all orders filled